<a href="https://colab.research.google.com/github/HedersonSantos/Noticias/blob/main/preProcessamento/preProcessamentoFSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import numpy as np
from google.colab import files, drive
from pathlib import Path
from sklearn.model_selection import train_test_split
import io, re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

### Montando Google Driver e abrindo arquivos de noticias

o arquivo articles.csv contém 167053 noticias da Folha de São Paulo. Este dataset está disponível em .... <br>
O arquivo news.csv contém xxx noticias extraídos dos sites UOL, G1, GoogleNews, Poder360 através de web scrapping cujo algoritmo foi montado por nós.

In [2]:
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Colab\ Notebooks/nlp_tcc
path = '/gdrive/My Drive/Colab Notebooks/nlp_tcc'
KEY='4/1AX4XfWjO8LFULyUVA2hr76Ekj281Rn58ieVYw_BkgE4HYvy9Lf8PJ1W3aOQ'

Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks/nlp_tcc


In [3]:
!rm *.zip
!ls

rm: cannot remove '*.zip': No such file or directory
articles.csv  news.csv	noticia_tratada.csv


In [4]:
#!unzip noticiasFSP.zip
#!ls *.csv


In [5]:
!ls *.csv -lh

-rw------- 1 root root 481M Jul 24 17:03 articles.csv
-rw------- 1 root root  93M Jul 24 15:58 news.csv
-rw------- 1 root root  93M Jul 24 15:42 noticia_tratada.csv


**Conjunto de Notícias da Folha de São Paulo**

In [6]:
dfFSP = pd.read_csv('articles.csv', header=0)
print(dfFSP.columns)
dfFSP.shape

Index(['title', 'text', 'date', 'category', 'subcategory', 'link'], dtype='object')


(167053, 6)

In [7]:
dfFSP.head(1)

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral","Com a possibilidade de uma condenação impedir sua candidatura em 2018, o ex-presidente Luiz Inácio Lula da Silva fez, nesta segunda (9), um discurso inflamado contra a Lava Jato, no qual disse saber que está ""lascado"", exigiu um pedido de desculpas do juiz Sergio Moro e afirmou que, mesmo fora da disputa pelo Planalto, será um cabo eleitoral expressivo para a sucessão de Michel Temer. Segundo o petista, réu em sete ações penais, o objetivo de Moro é impedir sua candidatura no ano que vem, desidratando-o, inclusive, no apoio a um nome alternativo, como o do ex-prefeito de São Paulo Fernando Haddad (PT), caso ele não possa concorrer à Presidência. ""Eu sei que tô lascado, todo dia tem um processo. Eu não quero nem que Moro me absolva, eu só quero que ele peça desculpas"", disse Lula durante um seminário sobre educação em Brasília. ""Eles [investigadores] chegam a dizer: 'Ah, se o Lula não for candidato, ele não vai ter força como cabo eleitoral'. Testem"", completou o petista. Para o ex-presidente, Moro usou ""mentiras contadas pela Polícia Federal e pelo Ministério Público"" para julgá-lo e condená-lo a nove anos e seis meses de prisão pelo caso do tríplex em Guarujá (SP). O ex-presidente disse ainda não ter ""medo"" dos investigadores que, de acordo com ele, estão acostumados a ""mexer com deputados e senadores"" que temem as apurações. ""Eu quero que eles saibam o seguinte: se eles estão acostumados a lidar com deputado que tem medo deles, a mexer com senadores que têm medo deles, quero dizer que tenho respeito profundo por quem me respeita, pelas leis que nós ajudamos a criar, mas não tenho respeito por quem não me respeita e eles não me respeitaram"", afirmou o petista. De acordo com aliados, Lula não gosta de discutir, mesmo que nos bastidores, a chance de não ser candidato ao Planalto e a projeção do nome de Haddad como plano B do PT tem incomodado os mais próximos ao ex-presidente. O ex-prefeito, que estava no evento nesta segunda, fez um discurso rápido, de menos de dez minutos, em que encerrou dizendo esperar que Lula assuma a Presidência em 2019. ""Espero que dia 1º de janeiro de 2019 esse pesadelo chamado Temer acabe e o senhor assuma a Presidência da República"", disse Haddad. 'DEMÔNIO DO MERCADO' Lula voltou a fazer um discurso mais agressivo em relação ao mercado e disse que ""não tem cara de demônio"", mas quer que o respeitem ""como se fosse"". ""Não tenho cara de demônio, mas quero que eles me respeitem como se eu fosse, porque eles sabem que a economia não vai ficar subordinada ao elitismo da sociedade brasileira"", disse o ex-presidente. O petista rivalizou ainda com o deputado Jair Bolsonaro (PSC-RJ), segundo colocado nas últimas pesquisas empatado com Marina Silva, e disse que se ele ""agrada ao mercado"", o PT tem que ""desagradar"". A Folha publicou nesta segunda (9) reportagem em que mostrou que o deputado ensaia movimento ao centro no debate econômico, adotando um discurso simpático aos investidores do mercado financeiro.",2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/1925743-lula-diz-que-esta-lascado-mas-que-ainda-tem-forca-como-cabo-eleitoral.shtml


**Quantidade de noticias por categoria**

In [8]:
#['title', 'text', 'date', 'category', 'subcategory', 'link'
categoriaFSP = dfFSP.groupby(['category'])['link'].count().reset_index().sort_values(by = 'link', ascending=False)

**Conjunto de Notícias da nossa base de dados**

In [9]:
dfNews = pd.read_csv('news.csv', header=0)
print(dfNews.columns)
dfNews.shape

Index(['FONTE', 'CATEGORIA', 'TEXTO', 'URL', 'PUBLISHEDAT', 'TITLE', 'AUTHOR',
       'DT_REGISTRO', 'TEXT_TRATADO'],
      dtype='object')


(11176, 9)

In [10]:
'''tornando o nome das colunas dos dois datasets iguais '''
dfNews = dfNews.rename(columns={'CATEGORIA':'category',
                       'URL':'link',
                       'PUBLISHEDAT':'date',
                       'TITLE':'title',
                       'TEXT_TRATADO':'text',
                       'FONTE':'fonte'}).copy()
dfNews = dfNews[['title', 'text', 'date', 'category', 'link','fonte']]


    

In [11]:
dfNews.head(1)

,title,text,date,category,link,fonte
0,"Putin não espera ""grandes avanços"" em cúpula com Biden","Em entrevista para as principais agências de notícias internacionais, incluindo a Efe, Putin enfatizou que os dois países, apesar das discordâncias, têm interesses em comum e que ele está confiante que a cúpula com Biden será construtiva.\nEntre as áreas de interesse comum, o chefe do Kremlin citou a solução de problemas ambientais, segurança estratégica e a dissolução de uma série de crises regionais.\n""Isso é muito importante, porque tem a ver com a segurança de nossos países"", afirmou Putin, além de destacar, por exemplo, que a crise no Oriente Médio ""leva ao crescimento do terrorismo"".\n""(Isso é) muito perigoso para todos, para os Estados Unidos, para a Rússia, para a Europa, para outras regiões do mundo"", acrescentou.\nPutin enfatizou a importância da segurança estratégica e garantiu que a Rússia não quer ""assustar ninguém"" com seus novos armamentos.\n""Sim, estamos desenvolvendo esses armamentos e conseguimos certos resultados"", disse o mandatário, para quem a Rússia está um passo ""à frente"" de outros países.\nEle também admitiu que os Estados Unidos e outras potências logo alcançarão resultados semelhantes, e portanto ""é melhor pensar em como vamos viver juntos"".\n\nO Conselho Tutelar de\nIvaiporã foi acionado por volta das 12h30 de sexta-feira (4) para acompanhar\nalgumas...\nO sábado (5) começa com tempo estável nas regiões paranaenses. Ao amanhecer, Apucarana e Arapongas registraram...\nO Senado vai analisar a Medida Provisória 1.034/2021, que aumenta a tributação de instituições financeiras O texto \nEm meio à pandemia da Covid-19, especialistas apontam que a menor frequência nos parques públicos pode estar...\nA Empresa de Limpeza Urbana de Salvador (Limpurb) fará uma série de ações até 11 de junho como parte da campanha...\nOs ministros das Finanças do G7 esperam anunciar um acordo neste sábado (5) sobre um imposto mínimo global e uma...",2021-06-05T03:00:00Z,mundo,https://noticias.uol.com.br/ultimas-noticias/efe/2021/06/05/putin-nao-espera-grandes-avancos-em-cupula-com-biden.htm,UOL


In [12]:
categoriaNossa = dfNews.groupby(['category'])['link'].count().reset_index().sort_values(by = 'link', ascending=False)

### Integrando as duas bases de dados de noticias

In [13]:
dfFSP['fonte']='FolhaSP'
dfnoticias = pd.concat([dfFSP,dfNews],sort=False)
dfnoticias.shape

(178229, 7)

In [14]:
dfnoticias['fonte'].value_counts()

FolhaSP           167053
UOL                 4034
google-news-br      2089
Poder360            1448
Notícias da TV      1262
Name: fonte, dtype: int64

In [15]:
deParaCategorias = pd.merge(categoriaFSP[['category','link']], categoriaNossa, on=['category'], how='outer')

**Categorias com mesmo nome nos dois conjunto de dados**

In [16]:
deParaCategorias[(~deParaCategorias['link_x'].isnull()) & (~deParaCategorias['link_y'].isnull()) ]

,category,link_x,link_y
1,colunas,21622.0,263.0
3,esporte,19730.0,590.0
4,mundo,17130.0,1251.0
5,cotidiano,16967.0,77.0
12,educacao,2118.0,27.0


**Categorias que estão no nosso dataset e não estão no dataset da FSP**

In [17]:
str(deParaCategorias[(deParaCategorias['link_x'].isnull()) ]['category'].tolist())

"['midia', 'local', 'politica', 'novelas', 'miscelanea', 'saude', 'economia', 'brasil', 'coronavirus', 'justica', 'celebridades', 'tecnologia', 'fantastico', 'meio-ambiente', 'podcast', 'eleicoes', 'loterias', 'casos-de-policia', 'olimpiadas', 'pesquisas', 'confere', 'enem', 'carros', 'agronegocios', 'pme', 'mais', 'tilt, reuters', 'portuguese', 'legislacao', 'atitude', 'internet', 'coluna', 'especiais', 'start', 'sociedade, educacao', 'realities', 'programas, 3-em-1', 'comprova', 'pix', 'combate', 'transito', 'nieman', 'motor', 'inovacao', 'historia', 'nossa', 'carros, colunas', 'boxe', 'tilt, estado', 'brics', 'mais, levante', 'sociedade, epoca', 'saude, estado', 'eurocopa', 'rio, rio', 'golfe', 'programas', 'carros, colunas, infomoto', 'justica, 3-dos-ex-pms-levados-por-salles-para-ministerio-foram-afastados-pelo-stf', 'exercito', 'diplomacia', 'redacao']"

In [18]:
str(categoriaFSP['category'].tolist())

"['poder', 'colunas', 'mercado', 'esporte', 'mundo', 'cotidiano', 'ilustrada', 'opiniao', 'paineldoleitor', 'saopaulo', 'tec', 'tv', 'educacao', 'turismo', 'ilustrissima', 'ciencia', 'equilibrioesaude', 'sobretudo', 'bbc', 'folhinha', 'empreendedorsocial', 'comida', 'asmais', 'ambiente', 'seminariosfolha', 'serafina', 'o-melhor-de-sao-paulo', 'vice', 'guia-de-livros-discos-filmes', 'topofmind', 'banco-de-dados', 'dw', 'especial', 'infograficos', 'cenarios-2017', 'rfi', 'guia-de-livros-filmes-discos', 'multimidia', 'treinamento', 'treinamentocienciaesaude', 'mulher', 'euronews', 'ombudsman', 'contas-de-casa', '2016', 'bichos', 'musica', '2015']"

DE/PARA de categorias.

nossa             FSP<br>
midia      <--    tv, multimidia, novelas, celebridades   <br>
local      <--    'saopaulo', 'o-melhor-de-sao-paulo' <br>
politica      <--    poder <br>
miscelanea -->     <br>
saude      <--   equilibrioesaude <br>
economia   <--   mercado <br>
brasil                   <br>
olimpiadas -->  esporte <br>
tecnologia  <-- tec,'banco-de-dados', 'dw'  <br>
meio-ambiente <-- ambiente <br>
enem --> educacao








In [19]:
dfnoticias['category'] = dfnoticias.loc[:,['category']].apply(lambda x: 'midia' if x['category'] in ['tv', 'multimidia', 'novelas', 'celebridades'] else 
                                                                        'local' if x['category'] in [ 'saopaulo', 'o-melhor-de-sao-paulo'] else 
                                                                        'politica' if x['category'] in ['poder'] else
                                                                        'saude' if x['category'] in ['equilibrioesaude'] else 
                                                                        'economia' if x['category'] in ['mercado'] else
                                                                        'esporte' if x['category'] in ['olimpiadas'] else
                                                                        'tecnologia' if x['category'] in [ 'tec','banco-de-dados', 'dw'] else
                                                                        'meio-ambiente' if x['category'] in [ 'ambiente'] else 
                                                                        'educacao' if x['category'] in ['enem'] else
                                                                        x['category'], axis=1)


**Quantitativo de noticias por categoria**

In [20]:
dfnoticias['category'].value_counts()

politica                                                                             23499
colunas                                                                              21885
economia                                                                             21476
esporte                                                                              20331
mundo                                                                                18381
cotidiano                                                                            17044
ilustrada                                                                            16345
local                                                                                 5628
midia                                                                                 4817
opiniao                                                                               4525
paineldoleitor                                                                        4011

In [90]:
dfnews_tratadas = dfnoticias.copy()
dfnews_tratadas['index_noticias'] = dfnoticias.index
idx=0
for index, row in dfnews_tratadas[dfnews_tratadas['fonte']=='UOL'][['fonte','category','text','index_noticias']].iterrows():
  print('********', row['category'], '********', row['fonte'], '********')
  print(row['text'])
  print("#####################################################################################################################")
  for match in re.finditer(r'(?s)((?:[^\n][\n]?)+)', row['text']):
   print(match.start(), '-', match.end(), 'index:', row['index_noticias'])
   if not ('...' in str(row['text'])[match.start(): match.end()]):
     #print(row['text'][match.start(): match.end()])
     dfnews_tratadas.loc[row['index_noticias'],['text']]=row['text'][match.start(): match.end()]
     print('foi')
  
  if idx == 0:
    break
  idx+=1

******** mundo ******** UOL ********
Em entrevista para as principais agências de notícias internacionais, incluindo a Efe, Putin enfatizou que os dois países, apesar das discordâncias, têm interesses em comum e que ele está confiante que a cúpula com Biden será construtiva.
Entre as áreas de interesse comum, o chefe do Kremlin citou a solução de problemas ambientais, segurança estratégica e a dissolução de uma série de crises regionais.
"Isso é muito importante, porque tem a ver com a segurança de nossos países", afirmou Putin, além de destacar, por exemplo, que a crise no Oriente Médio "leva ao crescimento do terrorismo".
"(Isso é) muito perigoso para todos, para os Estados Unidos, para a Rússia, para a Europa, para outras regiões do mundo", acrescentou.
Putin enfatizou a importância da segurança estratégica e garantiu que a Rússia não quer "assustar ninguém" com seus novos armamentos.
"Sim, estamos desenvolvendo esses armamentos e conseguimos certos resultados", disse o mandatário, 

In [88]:
dfnews_tratadas.loc[1182,'text']


1182    Partidos da base aliada intensificaram a pressão sobre o Palácio do Planalto com o início da tramitação da segunda denúncia da PGR (Procuradoria-Geral da República) contra o presidente Michel Temer (PMDB).  A conta ficou mais cara, já que os partidos que ajudaram o peemedebista a escapar da primeira denúncia, de corrupção passiva, agora querem a cabeça do ministro tucano Antonio Imbassahy (Secretaria de Governo), responsável pela articulação política do governo.  Eles também cobram a liberação mas rápida de verbas para emendas que apresentaram ao Orçamento —represadas, segundo líderes de vários partidos— e a entrega de cargos na máquina federal.  Alguns desses compromissos foram feitos pelo Planalto com o objetivo de barrar a primeira denúncia, congelada pela Câmara por 263 votos a 227 em agosto —era preciso o voto de pelo menos 342 deputados para que o Supremo Tribunal Federal fosse autorizado a analisar o caso.  Com isso, cresceu a desconfiança e a insatisfação de deputados, 

In [89]:
dfnoticias.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            11166, 11167, 11168, 11169, 11170, 11171, 11172, 11173, 11174,
            11175],
           dtype='int64', length=178229)